# 4. Core Concept of Spark

* Job: 코드들의 모음으로, 파일을 HDFS나 로컬에서 읽어오고, computation등을 한 후 write를 하는 작업을 실행하는 객체

* Stages: Job들은 여러개의 stage로 분리된다. spark는 map-reduce기반으로 작성되어 있으며, 각 stage는 map 또는 reduce로 분류된다. stage는 computational boundaries에 의해 각 computation(operator)로 분리되며, 해당 operator들은 single stage로 업데이트 될 수 없다.

* Tasks: 각 스테이지는 일부 tasks로 이루어져 있으며, 파티션별로 one task가 할당된다. 하나의 task는 하나의 파티션에서 하나의 machine에 의해서 수행된다.

* DAG: operation의 실행 순서는 DAG를 이용해 정의된다.

* Executor: task를 할당받은 machine

* Master: spark drive가 돌아가는 master node(machine)

* Slave: Executor program이 돌아가는(excutor) slave node

## Spark Components


![](images/2025-08-10-11-54-13.png)

1. Spark Driver
* process들을 나누고, SparkContext를 이용해 job을 스케쥴링 및 cluster manage를 합니다.

2. Executor
* 드라이버에 의해 스케쥴링된 각 task를 실행합니다.
* computation 결과를 메모리나 디스크에 저장합니다.
* storage system과 통신합니다

3. Cluter Manager
* Mesos
* YARN
* **Spark Standalone**

Spark Drvier는 아래와 같은 세부 컴포넌트로 나눠집니다.
![](images/2025-08-10-12-10-37.png)

* Spark Context
스파크 클러스터 간의 연결, RDD의 생성, 클러스터안의 변수 브로드캐스팅 및 축적을 나타냅니다.

* DAGScheduler
스테이지안의 job들의 DAG를 계산하고, TaskSceduler에게 각 job을 전달해줍니다. 이때 파일의 위치등을 고려하여 task의 적절한 location을 전달해줍니다.

* Task Sceduler
클러스터에게 task를 전달하고, 실행시키고, 실패 시 재시작등의 작업을 관리합니다.

* SchedulerBackend
task scheduler backend는 다양한 구현체가 존재합니다 (Mesos, YARN, Standalone, local)

* BlockManager
데이터 블록 단위의 인터페이스를 제공하며 다양한 저장장치(메모리, 디스크, off-heap)에서 block을 넣고 가져오는 등의 작업을 수행합니다.

## 4.4 How Spark Works?

![](images/2025-08-10-11-58-13.png)
스파크는 다양한 layer로 이루어져 있습니다.  각 layer는 서로 다른 책임을 가지고 있으며, layer끼리는 독립적입니다.

각 layer의 역할은 아래와 같습니다.

1. interpreter: spark는 scala interpreter를 약간 변형하여 구현되어있습니다. spark 콘솔에 입력한 코드(job)를 기반으로 operation graph를 생성합니다.

2. DAG Scehduler: DAG scheduler는 operation graph를 입력받아 수행됩니다. graph를 여러개의 map/reduce stage로 분할합니다. 각 스테이지는 여러개의 input data의 파티션들에 대한 tasks들로 이루어져 있습니다. DAG 스케쥴러는 operator들을 함께 묶어 하나의 스테이지로 optimize합니다. (e.g., 여러개의 map operator이 하나의 스테이지로 엮입니다.) 이 optimization이 spark 성능의 핵심입니다.

3. Task(Cluster) Scheduler: 각 stage들은 task scheduler에 의해 각 slave에게 task단위로 전달합니다. Task Scheduler는 stage간의 dependency에 대해 신경쓰지 않아도 됩니다.